# MacGyver MUD: Active Inference Essentials

## The Fast Track to Understanding Active Inference

**Version**: 1.0 "Essentials"

**Estimated Time**: 45-60 minutes

---

### What You'll Learn

You're locked in a room with a door (might be locked) and a window (escape route, but costly). How should an intelligent agent decide what to do?

This streamlined notebook covers:

1. **Expected Free Energy (EFE)** - How agents score actions
2. **Bayesian Belief Updates** - Learning from observations
3. **The Silver Gauge** - Pythagorean means reveal k≈0 for all simple skills
4. **Geometric Insights** - Why exploration-exploitation balance emerges naturally

### Structure

- **Interactive widgets** for hands-on exploration
- **Real-time visualizations** of belief updates
- **Checkpoints** to test understanding
- **Neo4j integration** for live data (optional)

---

*"The shortest path between two truths in the real domain passes through the complex domain." — Jacques Hadamard*

*We'll take the shortest path through Active Inference using geometry.*

In [ ]:
# Import all utilities
from macgyver_utils import *

# Standard notebook setup
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Imports loaded")
print("✓ Ready to explore Active Inference!")

In [ ]:
# Connect to Neo4j (optional - will use fallback data if unavailable)
initialize_neo4j(
    uri="bolt://localhost:7687",
    user="neo4j",
    password="macgyver123"
)

In [ ]:
# Display connection status
display(get_connection_status())

## Quick Start: The Challenge

You wake up in a locked room with:
- A **door** (unknown if locked or unlocked)
- A **window** (guaranteed escape, but difficult/costly)

Available skills:
- `peek_door` - Observe the door lock (costs time, gains info)
- `try_door` - Attempt to open door (costs more time, succeeds if unlocked)
- `go_window` - Climb out window (costly but guaranteed)

**The question**: What should you do first?

---

# Part 1: The Scenario

Let's explore the state space and understand our options.

## The Room Structure

The room exists as a state graph in our Neo4j database. Each action transitions between states, and some transitions depend on the door being locked or unlocked.

In [ ]:
# Visualize the room as a state graph
visualize_room_graph()

In [ ]:
# CHECKPOINT 1: Initial understanding quiz
create_quiz_widget(
    question="If you don't know whether the door is locked, which skill gives you the most INFORMATION?",
    options=[
        ('peek', 'peek_door - observe the lock'),
        ('try', 'try_door - attempt to open'),
        ('window', 'go_window - climb out window')
    ],
    correct_answer='peek',
    feedback_dict={
        'peek': 'Correct! Peeking is designed to gather information about the door state.',
        'try': 'Try_door also gives info (through success/failure), but it\'s more costly.',
        'window': 'The window doesn\'t tell you anything about the door.'
    }
)

## Understanding Beliefs

Since we don't know if the door is locked, we maintain a **belief distribution** over possible states:

- `P(locked)` = probability the door is locked
- `P(unlocked)` = 1 - P(locked)

This uncertainty drives our decision-making!

In [ ]:
# Interactive belief slider with recommendation
def update_belief_viz(change):
    with output:
        clear_output(wait=True)
        belief = change['new']
        plot_belief_distribution(belief)
        plt.show()
        show_recommendation(belief)

slider, output = create_belief_slider(update_belief_viz, initial_value=0.5)
display(slider, output)

# Trigger initial display
update_belief_viz({'new': 0.5})

In [ ]:
# Display skill properties as table
skills_df = query_crisp_skills()
display(HTML("<h3>Available Skills</h3>"))
display(skills_df.style.set_properties(**{'text-align': 'center'}).set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'center'), ('font-weight', 'bold')]}]
))

In [ ]:
# CHECKPOINT 2: Skills understanding
create_quiz_widget(
    question="Which skill has the highest GOAL value (most reward if successful)?",
    options=[
        ('peek', 'peek_door'),
        ('try', 'try_door'),
        ('window', 'go_window')
    ],
    correct_answer='try',
    feedback_dict={
        'peek': 'Peek has goal=0 because it doesn\'t directly achieve the escape goal.',
        'try': 'Correct! try_door has goal=10, the highest reward (if door is unlocked).',
        'window': 'go_window has goal=8, good but less than try_door.'
    }
)

## Part 1 Summary

Key insights:
- We face **uncertainty** about the door state
- Different skills offer different trade-offs: **cost vs goal vs information**
- Our beliefs influence which action is best

**Next**: How do we mathematically score actions under uncertainty?

---

# Part 2: Expected Free Energy

How do we choose the "best" action when outcomes are uncertain?

## The EFE Formula

Active Inference uses **Expected Free Energy (EFE)** to score actions:

```
EFE = Cost - Expected_Goal - Expected_Info
```

**Lower EFE is better** (less "surprise", more preferred)

Components:
- **Cost**: Resources consumed (time, energy)
- **Expected Goal**: Reward × P(success)
- **Expected Info**: Information gain (entropy reduction)

The agent chooses the action with **minimum EFE**.

## EFE in Detail

For `try_door`:
- Cost = 2.0 (time to try)
- Expected Goal = 10.0 × P(unlocked) = 10.0 × (1 - P(locked))
- Expected Info = 0.0 (trying doesn't reduce uncertainty, it resolves it)

For `peek_door`:
- Cost = 1.0
- Expected Goal = 0.0 (peeking doesn't achieve escape)
- Expected Info = 0.8 (95% accurate observation)

**The balance changes with belief!**

In [ ]:
# Interactive EFE calculator
cost_input = create_float_input('Cost:', 2.0, min_val=0)
goal_input = create_float_input('Goal Value:', 10.0, min_val=0)
prob_input = widgets.FloatSlider(
    value=0.5, min=0, max=1, step=0.05,
    description='P(success):',
    readout_format='.0%'
)
info_input = create_float_input('Info Gain:', 0.0, min_val=0)

calc_button = widgets.Button(description='Calculate EFE', button_style='info')
calc_output = widgets.Output()

def calculate_efe(b):
    with calc_output:
        clear_output()
        cost = cost_input.value
        goal = goal_input.value
        prob = prob_input.value
        info = info_input.value
        
        expected_goal = goal * prob
        efe = cost - expected_goal - info
        
        print(f"\nCalculation:")
        print(f"  Cost:          {cost:.2f}")
        print(f"  Expected Goal: {goal:.2f} × {prob:.2f} = {expected_goal:.2f}")
        print(f"  Expected Info: {info:.2f}")
        print(f"  " + "="*40)
        print(f"  EFE = {cost:.2f} - {expected_goal:.2f} - {info:.2f} = {efe:.2f}")
        print(f"\n{'Lower is better!' if efe < 0 else 'Higher cost than benefit'}")

calc_button.on_click(calculate_efe)

display(widgets.VBox([
    widgets.HTML("<h3>EFE Calculator</h3>"),
    cost_input, goal_input, prob_input, info_input,
    calc_button, calc_output
]))

# Auto-calculate on load
calculate_efe(None)

In [ ]:
# Demo: Score all skills at P(locked) = 50%
print("\n=" * 50)
print("SKILL SCORING DEMO: P(locked) = 50%")
print("=" * 50)
show_skill_scores(0.5)

In [ ]:
# CHECKPOINT 3: Calculate EFE by hand
create_quiz_widget(
    question="If P(locked) = 0.8, what is the EFE for try_door? (Cost=2, Goal=10, Info=0)",
    options=[
        ('zero', '0.0'),
        ('neg_six', '-6.0'),
        ('pos_zero', '+0.0'),
        ('pos_zero_two', '+0.2')
    ],
    correct_answer='pos_zero',
    feedback_dict={
        'zero': 'Close! Remember: EFE = Cost - (Goal × P(unlocked)) - Info',
        'neg_six': 'That would be if P(unlocked)=0.8. Remember P(unlocked)=1-P(locked)=0.2',
        'pos_zero': 'Correct! EFE = 2 - (10 × 0.2) - 0 = 2 - 2 = 0.0',
        'pos_zero_two': 'Close! Check your calculation of Expected Goal.'
    }
)

In [ ]:
# Interactive: How belief changes skill preference
def update_skill_scores(change):
    with scores_output:
        clear_output(wait=True)
        belief = change['new']
        show_skill_scores(belief)

belief_slider, scores_output = create_belief_slider(update_skill_scores, initial_value=0.5)

display(widgets.HTML("<h3>How Belief Affects Skill Selection</h3>"))
display(widgets.HTML("<p>Move the slider to see how skill preferences change with belief:</p>"))
display(belief_slider, scores_output)

# Trigger initial display
update_skill_scores({'new': 0.5})

In [ ]:
# Visualization: EFE across belief range
beliefs = np.linspace(0, 1, 100)
skills = [
    {'name': 'peek_door', 'cost': 1.0, 'goal': 0.0, 'info_gain': 0.8},
    {'name': 'try_door', 'cost': 2.0, 'goal': 10.0, 'info_gain': 0.0},
    {'name': 'go_window', 'cost': 5.0, 'goal': 8.0, 'info_gain': 0.0}
]

fig, ax = plt.subplots(figsize=(12, 6))

for skill in skills:
    efes = [score_skill(skill, b) for b in beliefs]
    ax.plot(beliefs, efes, label=skill['name'], linewidth=2.5)

ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5, label='Break-even')
style_plot(ax, 'EFE vs Belief: When to Explore vs Exploit', 
           'P(locked)', 'Expected Free Energy')
ax.legend(loc='best', fontsize=11)
ax.invert_yaxis()  # Lower is better
plt.tight_layout()
plt.show()

print("\n💡 Notice: At low P(locked), try_door dominates (exploitation)")
print("          At high P(locked), peek_door wins (exploration)")

<details>
<summary><strong>🔍 Math Deep-Dive: Why This Formula? (Click to expand)</strong></summary>

### The Free Energy Principle

EFE comes from the **Free Energy Principle** in neuroscience/AI:

**Key idea**: Agents minimize surprise (unexpected observations)

Free Energy is a computationally tractable upper bound on surprise:

```
F ≈ -ln P(observations | model)
```

For **future actions**, we can't observe yet, so we use **expected** free energy:

```
G(π) = E[Cost] - E[Goal Achievement] - E[Information Gain]
```

Where:
- **E[Cost]** = Pragmatic cost (energy, time)
- **E[Goal]** = Expected goal achievement (exploitation)
- **E[Info]** = Expected information gain (exploration)

This naturally balances **exploration** (learn about world) vs **exploitation** (achieve goals)!

### Connection to Control Theory

EFE is related to optimal control's "cost-to-go" but adds epistemic value (information).

</details>

## Key Insights: EFE

1. **EFE balances three factors**: cost, goal, and information
2. **Beliefs matter**: Same action has different EFE depending on what you believe
3. **Automatic exploration**: High uncertainty → information becomes valuable
4. **Automatic exploitation**: High certainty → go for the goal

**This is the core of Active Inference decision-making!**

## Part 2 Summary

We've learned:
- How to calculate EFE for any action
- Why beliefs change which action is preferred
- How exploration and exploitation emerge naturally

**Next**: How do beliefs update when we observe?

---

# Part 3: Bayesian Belief Updates

Actions change our beliefs. How?

## Bayes' Theorem for Belief Updates

When we observe something, we update our beliefs using **Bayes' Theorem**:

```
P(state | observation) = P(observation | state) × P(state) / P(observation)
```

In words:
- **Posterior** = (Likelihood × Prior) / Evidence

Example: If we peek and see "locked":
- Prior: P(locked) = 0.5
- Likelihood: P(see "locked" | actually locked) = 0.95
- Posterior: P(locked | saw "locked") = ?

Let's calculate!

## The Update Formula in Action

For our peek_door skill (95% accurate):

**If we observe "locked":**
```
P(locked | obs_locked) = [0.95 × P(locked)] / [0.95 × P(locked) + 0.05 × P(unlocked)]
```

**If we observe "unlocked":**
```
P(locked | obs_unlocked) = [0.05 × P(locked)] / [0.05 × P(locked) + 0.95 × P(unlocked)]
```

Notice: Observing "locked" increases P(locked), observing "unlocked" decreases it!

In [ ]:
# Interactive belief update simulator
prior_slider = widgets.FloatSlider(
    value=0.5, min=0, max=1, step=0.05,
    description='Prior P(locked):',
    readout_format='.0%',
    style={'description_width': '150px'}
)

true_state_select = widgets.Dropdown(
    options=[('Locked', 'locked'), ('Unlocked', 'unlocked')],
    value='locked',
    description='True State:',
    style={'description_width': '150px'}
)

action_select = widgets.Dropdown(
    options=[('Peek Door', 'peek_door'), ('Try Door', 'try_door')],
    value='peek_door',
    description='Action:',
    style={'description_width': '150px'}
)

sim_button = widgets.Button(description='Simulate Update', button_style='success', icon='play')
sim_output = widgets.Output()

def run_simulation(b):
    with sim_output:
        clear_output(wait=True)
        prior = prior_slider.value
        true_state = true_state_select.value
        action = action_select.value
        
        posterior, obs = simulate_belief_update(prior, true_state, action)
        
        plot_3panel_update(prior, action, obs, posterior)
        
        print(f"\n📊 Update Summary:")
        print(f"  Prior:      P(locked) = {prior:.0%}")
        print(f"  Action:     {action}")
        print(f"  Observed:   {obs}")
        print(f"  Posterior:  P(locked) = {posterior:.0%}")
        print(f"  Change:     {posterior - prior:+.0%}")

sim_button.on_click(run_simulation)

display(widgets.VBox([
    widgets.HTML("<h3>Bayesian Belief Update Simulator</h3>"),
    widgets.HTML("<p>Set parameters and click simulate to see belief updates:</p>"),
    prior_slider,
    true_state_select,
    action_select,
    sim_button,
    sim_output
]))

# Auto-run on load
run_simulation(None)

In [ ]:
# CHECKPOINT 4: Calculate posterior by hand
create_quiz_widget(
    question="If P(locked)=0.5 and we peek and observe 'locked', what's the new P(locked)? (95% accurate peek)",
    options=[
        ('fifty', '0.50 (no change)'),
        ('ninety', '0.90'),
        ('ninety_five', '0.95 (perfect certainty)'),
        ('seventy', '0.70')
    ],
    correct_answer='ninety_five',
    feedback_dict={
        'fifty': 'Beliefs should change with new evidence!',
        'ninety': 'Close! But with 50/50 prior and 95% accuracy, we get stronger update.',
        'ninety_five': 'Correct! P(locked|obs) = (0.95 × 0.5) / (0.95 × 0.5 + 0.05 × 0.5) = 0.95',
        'seventy': 'Too conservative. The observation is 95% reliable!'
    }
)

In [ ]:
# Full episode simulation
print("\n" + "="*60)
print("FULL EPISODE SIMULATION")
print("="*60)
print("\nScenario: Door is actually LOCKED, agent starts with P(locked)=0.5")
print("Let's watch Active Inference in action...\n")

episode = simulate_episode(initial_belief=0.5, true_door_state='locked', max_steps=5)

for step in episode:
    print(f"\n--- Step {step['step']} ---")
    print(f"Belief: P(locked) = {step['belief_locked']:.0%}")
    print(f"Scores: peek={step['scores']['peek_door']:.2f}, try={step['scores']['try_door']:.2f}")
    print(f"Action: {step['action']}")
    print(f"Result: {step['observation']}")
    print(f"New Belief: P(locked) = {step['new_belief']:.0%}")

print("\n" + "="*60)

In [ ]:
# Visualize episode progression
if len(episode) > 0:
    beliefs = [episode[0]['belief_locked']] + [s['new_belief'] for s in episode]
    steps = list(range(len(beliefs)))
    
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(steps, beliefs, marker='o', linewidth=3, markersize=10, color=COLORS['info'])
    
    # Annotate actions
    for i, step in enumerate(episode):
        ax.annotate(step['action'], 
                   xy=(i+0.5, (beliefs[i] + beliefs[i+1])/2),
                   ha='center', fontsize=9,
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))
    
    style_plot(ax, 'Belief Evolution Through Episode', 'Step', 'P(locked)')
    ax.set_ylim(-0.05, 1.05)
    ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.3, label='Maximum uncertainty')
    ax.legend()
    plt.tight_layout()
    plt.show()

## Interpreting the Episode

Watch what happens:

1. **Initial uncertainty** (P(locked) ≈ 0.5) → Agent chooses to **explore** (peek)
2. **After peeking**, belief updates dramatically (either ~0.95 or ~0.05)
3. **With high certainty**, agent switches to **exploitation** (try_door)

This is **Active Inference** in action:
- No hard-coded "explore first, then exploit"
- Behavior emerges from minimizing EFE
- Information-seeking is automatic when uncertain

## When to Peek vs Try?

The crossover point depends on the **EFE balance**:

- **High uncertainty** → Info gain is valuable → Peek wins
- **Low uncertainty** → Goal achievement dominates → Try wins

The threshold typically sits around **P(locked) ≈ 0.2-0.3** for these skill values.

**Key insight**: The decision threshold emerges from the math, not from rules!

## Part 3 Summary

We've learned:
- How Bayesian updates change beliefs based on observations
- How EFE + belief updates create explore-exploit behavior
- How to simulate complete episodes

**Next**: The geometric revelation that changes everything!

---

# Part 4: The Silver Gauge Revelation

We can score actions. We can update beliefs. But something's missing...

## What's Missing?

**Question**: Are our current skills well-balanced?

Consider:
- `peek_door`: ALL info, NO goal
- `try_door`: ALL goal, NO info
- `go_window`: ALL goal, NO info

These are **extreme specialists**! Is there a mathematical way to measure this imbalance?

Enter: **Pythagorean Means** (circa 500 BCE)

## Pythagorean Means: Ancient Math, Modern Insights

For two positive numbers a and b:

```
Harmonic Mean (HM)  = 2 / (1/a + 1/b)     [rewards balance]
Geometric Mean (GM) = √(a × b)             [multiplicative center]
Arithmetic Mean (AM) = (a + b) / 2         [average]
```

**Key property**: HM ≤ GM ≤ AM (always!)

**When equal**: Only when a = b (perfect balance)

**The closer GM is to AM, the more balanced a and b are!**

In [ ]:
# Interactive Pythagorean means calculator
a_input = widgets.FloatSlider(value=5.0, min=0.1, max=10, step=0.1, description='Value a:', readout_format='.1f')
b_input = widgets.FloatSlider(value=5.0, min=0.1, max=10, step=0.1, description='Value b:', readout_format='.1f')

means_button = widgets.Button(description='Calculate Means', button_style='info', icon='calculator')
means_output = widgets.Output()

def calc_means(b):
    with means_output:
        clear_output(wait=True)
        a = a_input.value
        b_val = b_input.value
        result = plot_pythagorean_means(a, b_val)
        if result:
            print(f"\n📐 Means Analysis:")
            print(f"  HM = {result['HM']:.3f}")
            print(f"  GM = {result['GM']:.3f}")
            print(f"  AM = {result['AM']:.3f}")
            print(f"\n  k = GM/AM = {result['k']:.3f}")
            print(f"  Balance: {result['balance']}")
            print(f"\n💡 k close to 1.0 means a and b are balanced!")

means_button.on_click(calc_means)

display(widgets.VBox([
    widgets.HTML("<h3>Pythagorean Means Explorer</h3>"),
    widgets.HTML("<p>Try different values to see how balance affects the means:</p>"),
    a_input, b_input,
    means_button,
    means_output
]))

# Auto-calculate
calc_means(None)

## The Inequality and k-Coefficient

Define the **balance coefficient**:

```
k = GM / AM
```

Properties:
- **k = 1.0**: Perfect balance (a = b)
- **k < 1.0**: Imbalance (the further from 1, the worse)
- **k → 0**: Extreme imbalance (one value near zero)

**This gives us a quantitative measure of balance!**

## The k≈0 Revelation

Now apply this to our skills:

**For exploration-exploitation balance**:
```
k_explore = GM(goal, info_gain) / AM(goal, info_gain)
```

Let's calculate for our skills:
- `peek_door`: goal=0, info=0.8 → k≈0 (IMBALANCED)
- `try_door`: goal=10, info=0 → k≈0 (IMBALANCED)

**All our simple skills are specialists with k≈0!**

This is not a bug—it's a **feature**. Simple skills are naturally specialists.

In [ ]:
# Calculate Silver Gauge for all crisp skills
silver_df = calculate_silver_for_crisp()

display(HTML("<h3>Silver Gauge Analysis: Crisp Skills</h3>"))
display(HTML("<p>k_explore measures goal vs info_gain balance (closer to 1.0 = more balanced):</p>"))

# Format for display
display_df = silver_df.copy()
display_df['k_explore'] = display_df['k_explore'].apply(
    lambda x: f"{x:.3f}" if not pd.isna(x) else "N/A"
)

display(display_df.style.set_properties(**{'text-align': 'center'}).set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'center'), ('font-weight', 'bold')]}]
))

print("\n🎯 Key Observation: k_explore is N/A (undefined) when goal=0 OR info=0")
print("   This means ALL simple skills are extreme specialists!")
print("\n💡 To get k≈1.0, we need skills with BOTH goal AND info_gain > 0")

## Results: Why k≈0 for Simple Skills

Mathematical reality:
```
If goal = 0:  GM = √(0 × info) = 0  →  k = 0/AM = 0
If info = 0:  GM = √(goal × 0) = 0  →  k = 0/AM = 0
```

**Implication**: You cannot have a balanced skill unless BOTH objectives are non-zero!

This means:
- Simple single-purpose skills are always specialists (k≈0)
- To get balance, you need **blended skills** (goal > 0 AND info > 0)

**This is the geometric diagnostic we needed!**

In [ ]:
# CHECKPOINT 5: Design challenge
create_quiz_widget(
    question="To create a skill with k_explore ≈ 0.9 (well-balanced), what properties should it have?",
    options=[
        ('high_goal', 'High goal, zero info_gain'),
        ('high_info', 'Zero goal, high info_gain'),
        ('both_high', 'Both goal and info_gain > 0, similar magnitudes'),
        ('both_zero', 'Both goal and info_gain = 0')
    ],
    correct_answer='both_high',
    feedback_dict={
        'high_goal': 'This gives k≈0 (specialist). Need BOTH objectives > 0.',
        'high_info': 'This gives k≈0 (specialist). Need BOTH objectives > 0.',
        'both_high': 'Correct! k = GM/AM is close to 1 when values are similar. Need goal≈info_gain.',
        'both_zero': 'This is undefined (0/0). Need positive values!'
    }
)

## Why This Works: The Geometric Insight

The **Silver Gauge** (k-coefficient from Pythagorean means) works because:

1. **Geometric mean rewards balance**: √(a × b) is maximized when a ≈ b
2. **Arithmetic mean is indifferent**: (a + b)/2 treats all combinations equally
3. **The ratio k = GM/AM captures the gap**: Perfect balance → k=1, imbalance → k→0

This is a **diagnostic metric**:
- Reveals what's missing (balanced skills)
- Quantifies the gap
- Guides design (create skills with k≈1)

**Ancient geometry solving modern AI problems!**

## Practical Implications

This geometric insight leads to:

1. **Skill Design**: Create "blended" skills that balance objectives
   - Example: `smart_try_door` with goal=7, info=6 → k≈0.98

2. **Multi-Objective Optimization**: Use k to measure trade-offs
   - Not just goal vs info, but any competing objectives
   - k_efficiency = balance of benefit vs cost

3. **Diagnostic-Driven Design**: 
   - Measure what exists (k≈0 for specialists)
   - Identify gaps (need k≈1 skills)
   - Design solutions (create balanced skills)

4. **Evolutionary Perspective**:
   - Simple skills are specialists (k≈0)
   - Evolution fills the Pareto front with balanced skills (k→1)
   - Diversity emerges from geometry!

## The Climax: Geometric Balance

**The full picture**:

```
Simple Skills (k≈0):           Blended Skills (k≈1):
├─ peek (info only)           ├─ smart_peek (goal=2, info=7)
├─ try (goal only)            ├─ careful_try (goal=8, info=5)
└─ window (goal only)         └─ observe_window (goal=6, info=6)

Diagnostic reveals gap  →  Design fills gap  →  Emergent diversity
```

**This is how Active Inference + Geometric Diagnostics = Innovation**

The math tells us:
- What we have (specialists)
- What we're missing (balanced skills)
- How to measure success (k→1)

**Geometry as design compass!**

## Part 4 Summary

We've discovered:
- Pythagorean means measure balance between objectives
- k-coefficient quantifies this balance (k=1 perfect, k→0 imbalanced)
- ALL simple skills have k≈0 (mathematical necessity)
- This diagnostic reveals the need for blended skills
- Geometry guides evolution toward diversity

**This is the Silver Gauge revelation: ancient math illuminating modern AI!**

---

# Part 5: Explore & Next Steps

Now that you understand the core concepts, let's explore the data!

## Neo4j Playground

If you're connected to Neo4j, you can run custom queries to explore the graph.

The database contains:
- **States**: Room states (stuck, escaped, etc.)
- **Skills**: Both crisp and blended skills
- **Transitions**: How skills move between states
- **Properties**: cost, goal, info_gain, k_explore, k_efficiency

Try the queries below to explore!

In [ ]:
# Interactive query widget
query_input = widgets.Textarea(
    value="MATCH (s:Skill) RETURN s.name, s.cost, s.goal, s.info_gain LIMIT 5",
    description='Cypher Query:',
    layout=widgets.Layout(width='100%', height='100px'),
    style={'description_width': '100px'}
)

query_button = widgets.Button(description='Run Query', button_style='primary', icon='database')
query_output = widgets.Output()

def run_custom_query(b):
    with query_output:
        clear_output()
        query = query_input.value
        
        if not NEO4J_CONNECTED:
            print("❌ Not connected to Neo4j. Connect first!")
            return
        
        print(f"Running query...\n")
        results = run_query(query)
        
        if results:
            df = pd.DataFrame(results)
            display(df)
            print(f"\n✓ Returned {len(results)} rows")
        else:
            print("No results returned (or query error)")

query_button.on_click(run_custom_query)

display(widgets.VBox([
    widgets.HTML("<h3>Neo4j Query Playground</h3>"),
    query_input,
    query_button,
    query_output
]))

## Try These Queries

**1. Find all blended skills (both goal and info_gain > 0):**
```cypher
MATCH (s:Skill)
WHERE s.goal > 0 AND s.info_gain > 0
RETURN s.name, s.goal, s.info_gain, s.k_explore
ORDER BY s.k_explore DESC
```

**2. Find most balanced skills (k_explore closest to 1):**
```cypher
MATCH (s:Skill)
WHERE s.k_explore IS NOT NULL
RETURN s.name, s.k_explore
ORDER BY s.k_explore DESC
LIMIT 5
```

**3. Explore state transitions:**
```cypher
MATCH (s:State)-[r]->(t:State)
RETURN s.name AS from, type(r) AS via, t.name AS to
```

**4. Find specialist skills (k_explore near 0 or NULL):**
```cypher
MATCH (s:Skill)
WHERE s.goal = 0 OR s.info_gain = 0
RETURN s.name, s.goal, s.info_gain, 'specialist' AS type
```

## Where to Go Deeper

Want to explore more? Check out:

### 1. The Full Deep Dive Notebook
- `MacGyverMUD_DeepDive.ipynb` (89 cells)
- Detailed mathematical derivations
- Multi-objective evolution
- Advanced visualizations

### 2. Extend the Code
- Design new blended skills
- Implement different belief update rules
- Add more state complexity
- Visualize Pareto fronts

### 3. Active Inference Literature
- Friston et al. (2015) - "Active Inference and Learning"
- Parr & Friston (2019) - "Working Memory, Attention, and Salience"
- Da Costa et al. (2020) - "Active Inference on Discrete State Spaces"

### 4. Pythagorean Means
- Classic inequality proofs
- Applications in economics, physics, engineering
- Generalized means (power means)

### 5. Multi-Objective Optimization
- Pareto efficiency
- NSGA-II and NSGA-III algorithms
- Hypervolume indicators

## Resources & Wrap-Up

### What We've Learned

1. **Active Inference**: Agents minimize Expected Free Energy
   - EFE = Cost - Expected_Goal - Expected_Info
   - Naturally balances exploration and exploitation

2. **Bayesian Updates**: Beliefs change with observations
   - P(state | obs) ∝ P(obs | state) × P(state)
   - Information reduces uncertainty

3. **Pythagorean Means**: Ancient geometry, modern insights
   - k = GM/AM measures balance
   - k≈0 for specialists, k≈1 for balanced skills

4. **Diagnostic-Driven Design**: Geometry guides evolution
   - Measure what exists
   - Identify gaps
   - Design solutions

### Core Insight

> **Simple skills are necessarily specialists (k≈0). To achieve balance (k≈1), you must blend objectives. This geometric diagnostic reveals the path to innovation.**

### Next Steps

- **Experiment**: Modify skills, change beliefs, run episodes
- **Explore**: Query the Neo4j database
- **Extend**: Design your own blended skills
- **Learn More**: Dive into the full notebook and literature

### Acknowledgments

This notebook builds on:
- Active Inference framework (Karl Friston et al.)
- Pythagorean mathematics (ancient Greeks)
- Multi-objective optimization theory
- Graph database technology (Neo4j)

---

**Thank you for exploring Active Inference with us!**

*"The only way to learn mathematics is to do mathematics." — Paul Halmos*

*Now go experiment!*